In [9]:
from tkinter import * # Carga módulo tk (widgets estándar)
import os, sys, subprocess
from tkinter import filedialog
from PyQt5.QtWidgets import QFileDialog
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
raiz = Tk() #Se crea el Framework o raiz principal
raiz.title("Data WareHouse") #Título de la barra superior de la ventana 
raiz.resizable(0,0) # (width,height) Para que no se pueda redimensionar la pantalla ni horizontal ni verticalmente
raiz.iconbitmap("logo.ico") # Para introducir el icono de barra superior izquierdo
#raiz.geometry("850x550") # Para fijar un tamaño de ventana(raiz). La raiz siempre se va a adaptar al tamaño de los contenedores que contenga, por eso lo dejamos comentado.
raiz.config(bg = "beige") # Fijar color de fondo

miFrame = Frame() # Se crea el Frame
miFrame.pack() # Se asigna el Frame al Framework 
#miFrame.pack(fill="both", expand = "True")
miFrame.config(bg = "powder blue") # Asignarle color de fondo
miFrame.config(width ="850", height = "550")

miFrame.config(bd=15) # Para indicar que queremos el borde más grueso
miFrame.config(relief="ridge") #Para cambiar el tipo borde

miFrame.config(cursor = "arrow") # Para cambiar el cursor  (cursor = "target") (cursor = "circle")

miLabel = Label(miFrame, text="Bienvenidos a Data WareHouse", fg="black", font=("Comic Sans MS", 18))
miLabel.place(x=10, y=20) 


miLabelCargar = Label(miFrame, text="", fg="black", font=("Comic Sans MS", 18))
miLabelCargar.place(x=350, y=120)
#miImagen = PhotoImage(file = "panda.jpg")
#miLabel2 = Label(miFrame, image=miImagen) 
#miLabel2.place(x=100, y=200)

miLabelGrafica1 = Label(miFrame, text="", fg="black", font=("Comic Sans MS", 18))
miLabelGrafica1.place(x=390, y=180)
        
# --------------------------------------------- Pantalla para Confirmar la Salida -------------------------------- #
"""class MyDialog:
    def __init__(self, parent):
        self.top = Toplevel(parent)
        self.parent = parent
        self.top.title("Salir")

        Label(self.top, text="¿Está seguro?").grid(row=0, column=0, columnspan=2)

        self.button1 = Button(self.top, text="Si, salir de la app.", command=self.salir)
        self.button2 = Button(self.top, text="No, solo minimizar.", command=self.minimizar)
        self.button1.grid(row=1, column=0, padx=5, pady=5)
        self.button2.grid(row=1, column=1, padx=5, pady=5)

    def salir(self):
        self.top.destroy()
        self.parent.destroy()

    def minimizar(self):
        self.top.destroy()
        self.parent.iconify()


class MyApp:
    def __init__(self, parent):
        self.parent = parent
        self.parent.protocol("WM_DELETE_WINDOW", self.on_closing)

    def on_closing(self):
        d = MyDialog(raiz)
        self.parent.wait_window(d.top)

app = MyApp(raiz)"""

# --------------------------------------------- Fin de Pantalla para Confirmar Salida -------------------------------- #

def CargarDatos(ruta):
    #print("Se van a cargar los datos:")
    # Esto funciona y abre el csv:
    #df = pd.read_csv('C:\\Users\\mdmar\\Desktop\\ficheroBueno.csv', sep=';',error_bad_lines=False, encoding='latin-1', header = 4)
    # Esto también funciona
    #df = pd.read_csv('ficheroBueno.csv', sep=';',error_bad_lines=False, encoding='latin-1', header = 4)
    df = pd.read_csv(ruta, sep=';',error_bad_lines=False, encoding='latin-1', header = 4)
    return df

def BotonCargar():
    raiz.fileName = filedialog.askopenfilename( filetypes=((".xls (No parseados)", "*.xls"),(".csv (Ya parseados)", "*.csv"),("Todos los ficheros","*.*")), title = "Seleccione un archivo")
    
    #print(raiz.fileName) # En "raiz.fileName" se almacena la ruta del archivo seleccionado (sin comillas). Por ej: C:/Users/mdmar/Desktop/ficheroBueno.csv
    nombreArchivo = raiz.fileName.split('/')[-1] # Se divide por "/" el string de la ruta y se obtiene la última posición con [-1]
    #aux = '"'+ nombreArchivo +'"' # Se entrecomilla el nombre del archivo
    #print(nombreArchivo)
    #ERROR: La ruta debería ser así: C:\Users\mdmar\Desktop\ficheroBueno.csv y la obtengo con las / (es decir, al revés)
    #fichero = open("C:/Users/mdmar/Desktop/datos.xls")
    df = CargarDatos(nombreArchivo)
    #print(df) # Muestra toda la información
    #print(df['Nombre y Apellidos'])  # Muestra columna deseada
    dframe = pd.DataFrame(df) # Se crea el DataFrame de los datos
    dframe.groupby(['Grupo','Descripción'])['Total'].sum().unstack('Grupo').plot(kind='barh', legend='Reverse', stacked =True)
    plt.xlabel('Total de alumnos matriculados')

def BotonGrafica1():
    print("Mostrar Gráfica 1")


def open_file(filename):   # Answer of stackoverflow
    if sys.platform == "win32":
        os.startfile(filename)
    else:
        opener = "open" if sys.platform == "darwin" else "xdg-open"
        subprocess.call([opener, filename])

        
""" Classs to create the main menu for set app parameters and show the help file"""
class GuiMenu:
    def __init__(self, master):
        self.master = master
        self.menu = Menu(master)
        master.config(menu=self.menu)

        self.submenu = Menu(self.menu)

        # ------------------ Sub Menu Configuración ------------------ #
        self.submenu_config = Menu(self.submenu)
        # Options of submenu
        self.submenu_config.add_command(label="1", command=self.set1)
        self.submenu_config.add_command(label="2", command=self.set2)
        self.submenu_config.add_command(label="3", command=self.set3)
        self.submenu_config.add_command(label="4", command=self.set4)
        self.submenu.add_cascade(label="Máx Gráficos", menu=self.submenu_config)


         # ------------------ Main Options of the Menu ------------------ #
        self.menu.add_cascade(label = "Configuración", underline = 0, menu = self.submenu)
        self.menu.add_command(label = "Ayuda",  command = self.open)
        self.menu.add_command(label="Acerca de...", command = self.openAbout)
        
    def open(self):
        open_file('ayuda.pdf') # Se abre el pdf de ayuda

    def openAbout(self):
        open_file('about.pdf') # Se abre el pdf de acerca de
        
    def set1(self):
        global max_p
        max_p = 1

    def set2(self):
        global max_p
        max_p = 2

    def set3(self):
        global max_p
        max_p = 3

    def set4(self):
        global max_p
        max_p = 4

        
    def log(self):
        x, y = get_window_pos(self.master)
        self.master.destroy()
        global ext_flag, log_flag
        ext_flag = False
        log_flag = True
        aneca_login(x, y)

    def extract(self):
        x, y = get_window_pos(self.master)
        self.master.destroy()
        global ext_flag, log_flag
        ext_flag = True
        log_flag = False
        get_only_info_window(x, y)

    def full_process(self):
        x, y = get_window_pos(self.master)
        self.master.destroy()
        global ext_flag, log_flag
        ext_flag = True
        log_flag = True
        info_window(x, y)
        
        
botonCargar = Button(miLabelCargar, text="Cargar Archivos", command = BotonCargar)
botonCargar.pack()


botonGrafica1 = Button(miLabelGrafica1, text="Gráfica 1", command = BotonGrafica1)
botonGrafica1.pack()

menu = GuiMenu(raiz) # Se llama a la clase anterior y se crea el menu.


raiz.mainloop() # Para que se ejecute continuamente 

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\mdmar\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-11-dd9903d83749>", line 86, in BotonCargar
    df = CargarDatos(nombreArchivo)
  File "<ipython-input-11-dd9903d83749>", line 74, in CargarDatos
    df = pd.read_csv(ruta, sep=';',error_bad_lines=False, encoding='latin-1', header = 4)
  File "C:\Users\mdmar\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 678, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\mdmar\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 440, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "C:\Users\mdmar\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 787, in __init__
    self._make_engine(self.engine)
  File "C:\Users\mdmar\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 1014, in _make_engine
    self._engine = CParserWrapper(self.f, **sel

#### Ventanas Bloqueantes (hasta que no contestas o cierras no puedes avanzar)

In [2]:
from tkinter.messagebox import *
#askokcancel("hola", "que tal todo")
"""if askquestion("pregunta", "¿quieres guardar los resultados?") == 'yes':
    print("has dicho que si")
else:
    print("has dicho que no")"""
#askretrycancel("title", "message")
#askyesno("title", "message") 

has dicho que no


In [2]:
#print(askfloat('Entry', 'Enter float'))  # Espera hasta que metes un decimal y te lo devuelve 
#print(askinteger('Entry', 'Enter integer')) # " "                    entero   "   "
#print(askstring('Entry', 'Enter string')) # "  "                     cadena    "  "

#### Ventanas que muestran un mensaje de error

In [10]:
#showerror("title", "message") # Muestra aspa roja
#showinfo("title", "message") # Muestra círculo azul de información 
#showwarning("title", "message") # Muestra triángulo amarillo

'ok'

#### Botón para GUARDAR un archivo (similar al de CARGAR)

In [12]:
#raiz.fileName = filedialog.asksaveasfilename( filetypes=((".pdf", "*.pdf"),(".tex (Latex)", "*.tex"),("Todos los ficheros","*.*")), title = "Guardar el archivo")


In [12]:
hoy = "dia"
print ('la fecha es "'+hoy+'"')

la fecha es "dia"


In [17]:
fichero = open("C:/Users/mdmar/Desktop/datos.xls")
print(fichero)

<_io.TextIOWrapper name='C:/Users/mdmar/Desktop/datos.xls' mode='r' encoding='cp1252'>
